In [1]:
import sys
sys.path.append('../modules')

from model import Resnet, Bottleneck
from utils import TinyImageNetDataset

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import numpy as np
import random
from pathlib import Path
import numpy as np 

In [2]:
# Tiny Imagenet

train_mean = (0.3975, 0.4481, 0.4802)
train_std = (0.2816, 0.2689, 0.2764)

In [3]:
transform_test = transforms.Compose([transforms.ToTensor(), transforms.Normalize(train_mean, train_std)])
testset = TinyImageNetDataset('./data/tiny_imagenet/tiny-imagenet-200', mode = 'val', transform=transform_test)

Preloading val data...:   0%|          | 0/10000 [00:00<?, ?it/s]

In [4]:
# param

batch_size = 100

In [5]:
test_dataloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)

In [6]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

model_save_path = './model'
model_list = sorted([x for x in Path(model_save_path).glob('*_epoch_*.pt')], key = lambda x : int(x.stem.strip('model_epoch_')))

best_model_path = model_list[81]

In [7]:
hit = 0
all_preds = []
all_labels = []

net = Resnet(Bottleneck, [3, 4, 6, 3], num_classes=200).to(device)
net = torch.nn.DataParallel(net, device_ids=[3,4,6,7]).to(device)
net.load_state_dict(torch.load(best_model_path)['state'])

criterion = nn.CrossEntropyLoss()

net.eval()

with torch.no_grad():
    test_loss = 0
    for (test_x, test_y) in tqdm(test_dataloader):
        preds = net(test_x.to(device))
        loss = criterion(preds, test_y.to(device))
        test_loss += loss
        hit += ((preds.argmax(dim=1) == test_y.to(device)).sum().item())

        all_preds.append(preds.argmax(dim=1))
        all_labels.append(test_y)

    accuracy = 100 * (hit/len(testset))
    print(f'accuracy : {accuracy:.4f}')
    print(f'val loss : {test_loss}')

    #del net

100%|██████████| 100/100 [00:19<00:00,  5.26it/s]

accuracy : 46.4800
val loss : 296.58758544921875
